In [ ]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
import pandas as pd

In [ ]:
qa = np.genfromtxt("uid_qa_test.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])
fe = np.genfromtxt("uid_pre_elim_test.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')
ftd.dtype.names = ('img_id', 'description')


In [ ]:
## Function- to convert sring into list.
## First convert strings to list of rows equal to rows in fe by making a function
## All image ids are one string in each row. By converting this string into list we will make each image ID an item in the list.

def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [ ]:
## convert strings in each row of pre-que to list of itmens.
## each row of all_x contains a list of image id(S) before question is asked and action like elimination/selection is done.

all_x = mk_list(fe['pre_que'])


In [ ]:
##check blanks
## create blank list
bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])


In [ ]:
## Now for each row in the list above, find description and add it
all_i_txt1 = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        #print(all_x.index)
        int_i_txt = []
        for img in all_x[rown]:
            if img in ftd['img_id']:
                d = str(ftd['description'][list(ftd['img_id']).index(img)])
                int_i_txt.append(str(d))
        original_ind.append(rown)
        all_i_txt1.append(int_i_txt)
        
            
            
            
            

In [ ]:
## add question and answer to each data point

all_i_txt = [row + [qa['que'][rn].strip(), qa['ans'][rn].strip()] for rn, row in zip(original_ind, all_i_txt1)]

    

In [ ]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer()

In [ ]:
## all similarities

all_sim = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])


In [ ]:
## This we need to know actually eliminated images so as to evaluate our model performance
## Secondly, this time we also peek the numbers to identify how many to put in each bucket- eliminated and non-eliminated

all_cheat_raw = mk_list(fe['curr_elim'])

## Pick only those which are not blank
all_cheat = [all_cheat_raw[i] for i in original_ind]


In [ ]:
## Sort all the above similarities with argsort which gives indices
## with peeking the length of target.

sorted_ind = []
for si, ln in zip(all_sim_q, all_cheat):
    y_top = si.argsort()[::-1][:len(ln)]
    sorted_ind.append(y_top)



In [ ]:
##clean_all_x
all_x_clean = [all_x[i] for i in original_ind]

In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [ ]:
pred_img = mk_imgid(zip(all_x_clean, sorted_ind))


In [ ]:
pre_all= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
rec_all = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img)]



In [ ]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]

In [ ]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))
pre_mean, rec_mean, fs_mean

In [ ]:
op = ["1", "TFIDF Test", "default", "Cosinesimilarity peek decision rule",str(pre_mean), str(rec_mean), str(fs_mean)]

In [ ]:
with open('Results with description.txt', 'a') as f:
    f.write('; '.join(op)+ '\n')